In [ ]:
%pwd

'/net/tscratch/people/plgkingak'

In [3]:
path = "weights2weights/weights_datasets"
device = "cuda"

In [2]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset

In [5]:
weight_dimensions = torch.load(f"{path}/weight_dimensions.pt")

In [6]:
dim_df = pd.DataFrame(weight_dimensions).T

In [7]:
dim_df

,0,1
base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.lora_A.weight,"(320,)","(1, 320)"
base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.lora_B.weight,"(320,)","(320, 1)"
base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_v.lora_A.weight,"(320,)","(1, 320)"
base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_v.lora_B.weight,"(320,)","(320, 1)"
base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_q.lora_A.weight,"(320,)","(1, 320)"
...,...,...
base_model.model.up_blocks.3.attentions.2.transformer_blocks.0.attn1.to_v.lora_B.weight,"(320,)","(320, 1)"
base_model.model.up_blocks.3.attentions.2.transformer_blocks.0.attn2.to_q.lora_A.weight,"(320,)","(1, 320)"
base_model.model.up_blocks.3.attentions.2.transformer_blocks.0.attn2.to_q.lora_B.weight,"(320,)","(320, 1)"
base_model.model.up_blocks.3.attentions.2.transformer_blocks.0.attn2.to_v.lora_A.weight,"(768,)","(1, 768)"


In [14]:
dim_df["use"] = 0
dim_df.loc[dim_df.index.str.contains("mid_block"), "use"] = 1

In [15]:
dim_df.use.sum()

8

In [22]:
dim_df["lengths"] = [a.numel() for a,_ in list(weight_dimensions.values())]

In [25]:
dim_df["start"] = dim_df.lengths.cumsum().shift(fill_value=0)
dim_df["end"] = dim_df.start + dim_df.lengths

In [28]:
dim_df[dim_df.use == 1]

,0,1,use,lengths,start,end
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn1.to_q.lora_A.weight,"(1280,)","(1, 1280)",1,1280,35968,37248
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn1.to_q.lora_B.weight,"(1280,)","(1280, 1)",1,1280,37248,38528
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn1.to_v.lora_A.weight,"(1280,)","(1, 1280)",1,1280,38528,39808
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn1.to_v.lora_B.weight,"(1280,)","(1280, 1)",1,1280,39808,41088
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn2.to_q.lora_A.weight,"(1280,)","(1, 1280)",1,1280,41088,42368
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn2.to_q.lora_B.weight,"(1280,)","(1280, 1)",1,1280,42368,43648
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn2.to_v.lora_A.weight,"(768,)","(1, 768)",1,768,43648,44416
base_model.model.mid_block.attentions.0.transformer_blocks.0.attn2.to_v.lora_B.weight,"(1280,)","(1280, 1)",1,1280,44416,45696


In [35]:
indices = np.concatenate([np.arange(row.start, row.end) for _, row in dim_df[dim_df.use == 1].iterrows()])
len(indices)


9728

In [37]:
weights = torch.load(f"{path}/identities/all_weights.pt", torch.device(device))

In [ ]:
torch.save(weights[:,indices], f'{path}/mid_block.pt')

In [4]:
dataset = torch.load(f'{path}/mid_block.pt', torch.device(device))

In [5]:
Dataset.from_dict({"data":dataset}).save_to_disk(f'{path}/mid_block')

Saving the dataset (0/6 shards):   0%|          | 0/64974 [00:00<?, ? examples/s]

In [8]:
d = Dataset.from_dict({"data":dataset})

In [9]:
d

Dataset({
    features: ['data'],
    num_rows: 64974
})

In [15]:
len(d[0]["data"])

9728